In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def clean_text(text):

    # Change the whole text to uniform case (lower)
    text = str(text).lower()

    # Remove punctuation and numbers
    cleaned = ""
    for char in text:
        if char.isalpha() or char.isspace():
            cleaned += char
        else:
            cleaned += " "

    # Remove extra spaces
    words = cleaned.split()
    cleaned_text = " ".join(words)

    return cleaned_text

def detect_spam():

    # Load the CSV file
    data = pd.read_csv('spam_ham_dataset.csv', on_bad_lines = 'skip', engine = 'python')


    # Get the columns
    columns = data.columns.tolist()
    if len(columns) < 4:
        print(f"ERROR: Expected 4 columns, but found {len(columns)}")
        print("Please check your CSV file structure.")
        return None, None

    text_col = columns[2]
    label_col = columns[3]

    # Clean the data
    data = data.dropna()

    # Prepare the data
    X = data[text_col].apply(clean_text)
    y = data[label_col]

    print(f"\nTotal emails: {len(X)}")
    print(f"Spam emails: {sum(y)}")
    print(f"Ham emails: {len(y) - sum(y)}")

    # Split data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convert text to numbers using TF-IDF
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Train two simple models for finding best one
    models = {
        'Naive Bayes': MultinomialNB(),
        'Logistic Regression': LogisticRegression(random_state=42)
    }

    results = {}

    for name, model in models.items():
        # Train the model
        model.fit(X_train_tfidf, y_train)

        # Make predictions
        predictions = model.predict(X_test_tfidf)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, predictions)
        results[name] = {'model': model, 'accuracy': accuracy}

        print(f"\n{name}:")
        print(f"Accuracy: {accuracy:.4f}")

    # Choose the best model
    best_model_name = max(results, key=lambda x: results[x]['accuracy'])
    best_model = results[best_model_name]['model']
    best_accuracy = results[best_model_name]['accuracy']

    print(f"\nBest Model: {best_model_name} (Accuracy: {best_accuracy:.4f})")

    # Test on entire dataset
    X_all_tfidf = vectorizer.transform(X)
    all_predictions = best_model.predict(X_all_tfidf)
    overall_accuracy = accuracy_score(y, all_predictions)

    # Show confusion matrix
    cm = confusion_matrix(y, all_predictions)
    print(f"\nConfusion Matrix:")
    print(f"True Ham predicted as Ham: {cm[0,0]}")
    print(f"True Ham predicted as Spam: {cm[0,1]}")
    print(f"True Spam predicted as Ham: {cm[1,0]}")
    print(f"True Spam predicted as Spam: {cm[1,1]}")

    # Function to predict new emails
    def predict_new_email(email_text):
        """Predict if a new email is spam (1) or ham (0)"""
        cleaned = clean_text(email_text)
        email_tfidf = vectorizer.transform([cleaned])
        prediction = best_model.predict(email_tfidf)[0]
        probability = best_model.predict_proba(email_tfidf)[0]

        print(f"\nEmail: {email_text}")
        print(f"Prediction: {prediction} ({'Spam' if prediction == 1 else 'Ham'})")
        print(f"Confidence: {max(probability):.3f}")
        return prediction

    return best_model, vectorizer, predict_new_email

# Run the spam detection
if __name__ == "__main__":
    model, vectorizer, predict_function = detect_spam()



Total emails: 5171
Spam emails: 1499
Ham emails: 3672

Naive Bayes:
Accuracy: 0.9304

Logistic Regression:
Accuracy: 0.9739

Best Model: Logistic Regression (Accuracy: 0.9739)

Confusion Matrix:
True Ham predicted as Ham: 3613
True Ham predicted as Spam: 59
True Spam predicted as Ham: 30
True Spam predicted as Spam: 1469
